In [3]:
import numpy as np 
import pandas as pd
import re

# Extension Processing and Categorization in WDI Indicator Data

## Objective
The goal of this notebook is to process and categorize the **extensions** found in the WDI (World Development Indicators) dataset. Extensions provide additional details such as **gender, demographic groups, education level, and geographic location**. By extracting and structuring these extensions into separate columns ( or jsut one column), we aim to enhance the hierarchical organization of the data for better analysis and visualization.

## Challenges Encountered when analysing the data 
1. **Missing Extensions**: Some indicators in the WDI dataset lack extensions, making it difficult to determine additional attributes for certain data points.  
2. **Ambiguous Extensions**: Certain extensions, such as **"FE"**, do not always correspond to a single category (e.g., "FE" may indicate **Female** in some cases but represent something else in other contexts). This requires additional context or mapping strategies to ensure accurate classification.  




In [ ]:
codes_dict = {

        "Gender": ["FE", "FM", "MA", "MF", "HI"],
        "Age": ["14", "YG", "OL"],
        "Population": ["P1", "P2", "P3", "P6", "Q1", "Q5", "R1", "R2", "R3", "R4", "R5", "R6"]


        "Education Level": ["BA", "DO", "MS", "UP", "ST", "VO", "GC"],
        "Literacy": ["LT", "IL"],
        "Other Education-Related": ["TE", "TC", "EC"]


        "Employment and Labor": ["EM", "TF", "HR", "PU"],
        "Economic Measures": ["GDP", "GNI", "GN", "VE", "RV"],
        "Social Behavior": ["SB", "SS"],
        "Sector and Industry": ["AG", "CL", "FI", "EC", "SV", "SC"]


        "Energy": ["CO", "EI", "FO", "NG", "KH"],
        "Environmental": ["CE", "GF", "HD", "LU"]

        "Income and Wealth": ["HI", "GD", "GE", "GP"],
        "Health and Well-being": ["HL", "DH"],
        "Government and Public Sector": ["PB", "GT"],
        "Research and Development": ["RD"]
    
}
